<img src="imgs/logo-spegc.svg" width=30%>

# Sistemas de recomendación

Los algoritmos de machine learning en los sistemas de recomendación se clasifican generalmente en dos categorías: los **métodos basados en contenido** y los **métodos de filtrado colaborativo**, aunque los recomendadores modernos combinan ambos enfoques.

Los métodos basados en el contenido hacen uso de la similitud de los atributos de los elementos vistos por el usuario anteriormente para recomendar unos parecidos. Los métodos de colaborativos calculan la similitud entre usuarios para recomendar.

## Sistemas basados en contenidos

Se recomiendan contenidos similares a los que el usuario ha consumido anteriormente. Pero, ¿cómo saber la similitud que existe entre dos productos? Supongamos una base de datos de películas. Estas tienen género, año, director, actores... y demás propiedades que pueden conformar un vector de caraterísticas. Por tanto, sería posible establecer métricas. Entre ellas podemos tener:

#### Distancia euclídea

$$ similitud(\textbf{a},\textbf{b}) = \sqrt{\sum_i {(a_i - b_i)^2}}$$


#### Coseno del ángulo

$$ similitud(\textbf{a},\textbf{b}) = cos(\theta) = \frac {\textbf{a} \cdot \textbf{b}}{  \left\| \textbf{a} \right\| \cdot \left\| \textbf{b} \right\|    }$$

Es relativamente fácil establecer similitudes entre vectores de características, pero ¿cómo hallar, por ejemplo, similitudes entre descripciones textuales de un artículo? 

### TF-IDF

Tf-idf (del inglés Term frequency – Inverse document frequency), **frecuencia de término – frecuencia inversa de documento** (o sea, la frecuencia de ocurrencia del término en la colección de documentos), es una medida numérica que expresa cuán relevante es una palabra para un documento en una colección.

Para la **frecuencia de término** $tf(t, d)$, la opción más sencilla es usar la frecuencia bruta del término $t$ en el documento $d$, o sea, el número de veces que el término $t$ ocurre en el documento $d$, aunque hay variantes, como la frecuencia de término normalizada.

$${\displaystyle \mathrm {tf} (t,d)={\frac {\mathrm {f} (t,d)}{\max\{\mathrm {f} (t,d):t\in d\}}}}$$

La **frecuencia inversa de documento** es una medida de si el término es común o no, en la colección de documentos. Se obtiene dividiendo el número total de documentos por el número de documentos que contienen el término, y se toma el logaritmo de ese cociente:

$${\displaystyle \mathrm {idf} (t,D)=\log {\frac {|D|}{|\{d\in D:t\in d\}|}}}$$

El índice **tf-idf** se calcula como el producto de los dos factores anteriores.

$${\displaystyle \mathrm {tfidf} (t,d,D)=\mathrm {tf} (t,d)\times \mathrm {idf} (t,D)}$$

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names())
print(X)
print(X.shape)

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
  (0, 8)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045
  (1, 8)	0.281088674033753
  (1, 3)	0.281088674033753
  (1, 6)	0.281088674033753
  (1, 1)	0.6876235979836938
  (1, 5)	0.5386476208856763
  (2, 8)	0.267103787642168
  (2, 3)	0.267103787642168
  (2, 6)	0.267103787642168
  (2, 0)	0.511848512707169
  (2, 7)	0.511848512707169
  (2, 4)	0.511848512707169
  (3, 8)	0.38408524091481483
  (3, 3)	0.38408524091481483
  (3, 6)	0.38408524091481483
  (3, 2)	0.5802858236844359
  (3, 1)	0.46979138557992045
(4, 9)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


## Similitud basada descripciones textuales

Leemos el fichero de metadatos de las películas, de donde extraeremos sus sinopsis.

In [22]:
%matplotlib inline
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

md = pd.read_csv('data/the-movies-dataset/movies_metadata.csv', low_memory=False)

Leemos el fichero de tags y nos quedamos solo con las películas que tienen tags.

In [23]:
links_small = pd.read_csv('data/the-movies-dataset/links_small.csv')
print(links_small.columns.values)
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int') # Tags 

md = md.drop([19730, 29503, 35587]) # We have to remove several rows without id

md['id'] = md['id'].astype('int')
smd = md[md['id'].isin(links_small)] # We'll work only with a subset of rows, just for the sake of speed
#smd = md
print("Tenemos " + str(smd.shape[0]) + " películas y " + str(smd.shape[1]) + " variables.")

['movieId' 'imdbId' 'tmdbId']
Tenemos 9099 películas y 24 variables.


Unimos 'overwiew' y 'tagline'

In [ ]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [20]:
print(smd['tagline'][3])

Friends are the people who let you be yourself... and never let you forget it.


In [ ]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

print(tfidf_matrix.shape)

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

print(cosine_sim[:5,:5])
print(cosine_sim.shape)

smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])


def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

Probemos nuestro recomendador

In [24]:
get_recommendations('The Hobbit').head(10)

8431                  The Hobbit: The Desolation of Smaug
8188                    The Hobbit: An Unexpected Journey
8723            The Hobbit: The Battle of the Five Armies
3865    The Lord of the Rings: The Fellowship of the Ring
8016                                        Mirror Mirror
3896                                         Dragonslayer
5135                                 The Ten Commandments
4993                                            Foul Play
7809                                        Your Highness
737                                      The Wizard of Oz
Name: title, dtype: object

## Sistemas colaborativos

Computan la similitud entre usuarios. Por ejemplo, si vamos a recomendar un grupo musical al usuario U, buscaremos sus $k$ vecinos más próximos y, de los grupos que les hayan gustado a esos $k$ vecinos, recomendaremos los que U aún no haya escuchado.

|           | Pink Floyd | The Who | U2 | Aerosmith | Manu Carrasco | Bisbal | Miles Davis | John Coltrane |   |   |
|-----------|------------|---------|----|-----------|---------------|--------|-------------|---------------|---|---|
| Usuario 1 | 1          | 1       | 0  | 1         | 0             | 0      | 0           | 0             |   |   |
| Usuario 2 | 0          | 0       | 0  | 0         | 1             | 1      | 0           | 0             |   |   |
| Usuario 3 | 1          | 0       | 1  | 1         | 0             | 0      | 0           | 1             |   |   |

El algoritmo más simple calcula la similitud de coseno o correlación de filas (usuarios) y recomienda elementos que hayan gustado a los $k$ vecinos más cercanos.

## Ejemplo de filtrado colaborativo

Tenemos un conjunto de usuarios y valoraciones que hacen esos usuarios sobre películas que han visto. Lo expresamos en forma de diccionario.

In [26]:
# A dictionary of movie critics and their ratings of a small
# set of movies

critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,
'The Night Listener': 3.0},
         
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 3.5},
         
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
'Superman Returns': 3.5, 'The Night Listener': 4.0},
         
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
'The Night Listener': 4.5, 'Superman Returns': 4.0,
'You, Me and Dupree': 2.5},
         
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 2.0},
         
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
         
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

### Buscando similitudes entre usuarios

Tenemos principalmente dos formas de definir la similitud entre dos personas: distancia euclídea y correlación de Pearson (https://es.wikipedia.org/wiki/Coeficiente_de_correlaci%C3%B3n_de_Pearson).

#### Distancia euclídea

In [27]:
# Returns a distance-based similarity score for person1 and person2
def sim_distance(prefs,person1,person2):
    # Get the list of shared_items
    si={}
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1
    # if they have no ratings in common, return 0
    if len(si)==0: return 0
    
    # Add up the squares of all the differences
    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2)
        for item in prefs[person1] if item in prefs[person2]])
    return 1/(1+sum_of_squares)

sim_distance(critics,'Lisa Rose','Gene Seymour')

0.14814814814814814

#### Correlación de Pearson

Con la correlación de Pearson tratamos de evitar que la "generosidad" de la puntuación dada por uno y otro usuario afecte a la medida de sus gustos.

$${\displaystyle \rho _{X,Y}={\sigma _{XY} \over \sigma _{X}\sigma _{Y}}={E[(X-\mu _{X})(Y-\mu _{Y})] \over \sigma _{X}\sigma _{Y}},}$$

- ${\sigma _{XY}}$ es la covarianza de ${(X,Y)}$
- ${\sigma _{X}}$ es la desviación estándar de la variable ${X}$.
- ${\sigma _{Y}}$ es la desviación estándar de la variable ${Y}$.

<img src="images/pearson.png" width=50%>

La fórmula alternativa que usaremos para calcular esta correlación será:

$${\displaystyle r_{xy}={\frac {\sum x_{i}y_{i}-n{\bar {x}}{\bar {y}}}{(n-1)s_{x}s_{y}}}={\frac {n\sum x_{i}y_{i}-\sum x_{i}\sum y_{i}}{{\sqrt {n\sum x_{i}^{2}-(\sum x_{i})^{2}}}~{\sqrt {n\sum y_{i}^{2}-(\sum y_{i})^{2}}}}}.}$$

In [28]:
from math import sqrt 

# Returns the Pearson correlation coefficient for p1 and p2
def sim_pearson(prefs,p1,p2):
    # Get the list of mutually rated items
    si={}
    for item in prefs[p1]:
        if item in prefs[p2]: si[item]=1

    # Find the number of elements
    n=len(si)
    
    # if they are no ratings in common, return 0
    if n==0: return 0

    # Add up all the preferences
    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])

    # Sum up the squares
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])
    
    # Sum up the products
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
    
    # Calculate Pearson score
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0: return 0
    r=num/den
    return r

print(sim_pearson(critics,'Lisa Rose','Gene Seymour'))

0.39605901719066977


Veamos quienes son los usuarios más parecidos a Toby.

In [29]:
# Returns the best matches for person from the prefs dictionary.
# Number of results and similarity function are optional params.
def topMatches(prefs,person,n=5,similarity=sim_pearson):
    scores=[(similarity(prefs,person,other),other) for other in prefs if other!=person]
        
    # Sort the list so the highest scores appear at the top
    scores.sort( )
    scores.reverse( )
    return scores[0:n]

topMatches(critics,'Toby',n=3)

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig')]

## Ranking items

Aunque tuviéramos una persona con gustos muy parecidos a los nuestros, no deberíamos fijarnos solamente en sus recomendaciones, estaríamos muy limitados a sus gustos. Deberíamos pesar sus opiniones con los demás usuarios que pudieran recomendarme otras cosas.

<img src="images/rank-items.png" width=80%>

In [30]:
# Gets recommendations for a person by using a weighted average
# of every other user's rankings
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simSums={}
    for other in prefs:
        # don't compare me to myself
        if other==person: continue
        sim=similarity(prefs,person,other)
        
        # ignore scores of zero or lower
        if sim<=0: continue
        for item in prefs[other]:
        
            # only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item]==0:

                # Similarity * Score
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim

                # Sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+=sim

    # Create the normalized list
    rankings=[(total/simSums[item],item) for item,total in totals.items( )]

    # Return the sorted list
    rankings.sort( )
    rankings.reverse( )
    return rankings

getRecommendations(critics,'Toby')

[(3.3477895267131017, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.530980703765565, 'Just My Luck')]